<a href="https://colab.research.google.com/github/sarabert96/Colexification/blob/main/check/bootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
!gdown --id 1a-Bnhiwo3TArMAKKABETdie4flLTF5C1
df_colex = pd.read_csv('df_colexifications.csv')

Downloading...
From: https://drive.google.com/uc?id=1a-Bnhiwo3TArMAKKABETdie4flLTF5C1
To: /content/df_colexifications.csv
7.64MB [00:00, 66.9MB/s]


apply multi-word and remove old/middle... varieties

In [3]:
!gdown --id 1EySPG8ZDMfMTvhUQaw8w1k6NgywI4N4S #df_all_raw
df_all = pd.read_csv('df_all_raw.csv')

Downloading...
From: https://drive.google.com/uc?id=1EySPG8ZDMfMTvhUQaw8w1k6NgywI4N4S
To: /content/df_all_raw.csv
244MB [00:01, 184MB/s]


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [40]:
df_all.groupby('Glottocode').size()

Glottocode
abag1245     108
abam1239      95
abau1245      90
abaz1241      77
abkh1244    1354
            ... 
zima1244     460
zinz1238     896
zizi1238     419
zuni1245     981
zuoj1238    1690
Length: 2279, dtype: int64

In [41]:
df_all.groupby('variety').size()

variety
'Are'are (1)    327
'Are'are (2)    327
A'jië            88
Abaga           108
Abau             90
               ... 
yabong          327
yangulam        274
yarawata        343
yoidik          323
Ömie            145
Length: 3050, dtype: int64

we saw that the number of glottocode is different from the number of variety! problem when merge? problem when inner join on R?

create new df with family, glottocode and variety info to be joined into colexification df.


PS: this part can be avoided if / when we modify the R code

In [4]:
df_support = df_all[['Family','variety','Glottocode']].copy()

In [5]:
df_support=df_support.drop_duplicates()
df_support

,Family,variety,Glottocode
0,Sino-Tibetan,Bugun Bichom,bich1234
1,Sino-Tibetan,Bugun Kaspi,kasp1234
2,Sino-Tibetan,Bugun Namphri,namp1239
3,Sino-Tibetan,Bugun Singchung,sing1271
4,Sino-Tibetan,Bugun Wangho,wang1301
...,...,...,...
1358018,Nuclear Trans New Guinea,yaben,yabe1255
1358019,Nuclear Trans New Guinea,yabong,yabo1240
1358020,Nuclear Trans New Guinea,yangulam,yang1298
1358021,Nuclear Trans New Guinea,yarawata,yara1250


In [6]:
# join df_colex with family and variety info
df_colex_ok = pd.merge(df_colex, df_support, on="Glottocode")

**knowing the data**: 185 linguistic fam, fam with most lang and with least

In [7]:
df_colex_ok.groupby('Family').size()

Family
Abkhaz-Adyge     508
Abun              13
Afro-Asiatic    5021
Aikanã            22
Ainu             123
                ... 
Yuat              10
Yukaghir         254
Yámana           203
Zamucoan         250
Zuni             189
Length: 185, dtype: int64

In [8]:
# df_fam --> df with families and varieties
df_fam = df_colex_ok[['Family','variety']].copy()
dic_fam=Counter(df_fam['Family'])
# print the family with most languages
print('family with max variety ', dic_fam.most_common(1))
# print family with least languages
print('family with min variety ',dic_fam.most_common()[-1])
df_fam = df_fam.reset_index(drop=True)


family with max variety  [('Nakh-Daghestanian', 51736)]
family with min variety  ("West Bird's Head", 1)


Add concepticon_pair column for future purposes

In [9]:
df_colex_ok['Concepticon_pair']=list(zip(df_colex_ok['Concepticon_Gloss.x'],df_colex_ok['Concepticon_Gloss.y']))

In [10]:
df_colex_ok

,clics_form,Concepticon_ID.x,Glottocode,Concepticon_Gloss.x,Concepticon_ID.y,Concepticon_Gloss.y,colex,Family,variety,Concepticon_pair
0,s@,1369,hrus1242,GOLD,1927,BAMBOO,1,Hruso,Hruso Aka Jamiri,"(GOLD, BAMBOO)"
1,s@,1369,hrus1242,GOLD,946,BLOOD,1,Hruso,Hruso Aka Jamiri,"(GOLD, BLOOD)"
2,dzju,1425,hrus1242,GREEN,1424,YELLOW,1,Hruso,Hruso Aka Jamiri,"(GREEN, YELLOW)"
3,fu,634,hrus1242,MEAT,1174,YESTERDAY,1,Hruso,Hruso Aka Jamiri,"(MEAT, YESTERDAY)"
4,phu,639,hrus1242,MOUNTAIN,1077,WHEAT,1,Hruso,Hruso Aka Jamiri,"(MOUNTAIN, WHEAT)"
...,...,...,...,...,...,...,...,...,...,...
312563,sopos,1455,pila1246,PULL,1917,TIE,1,Nuclear Trans New Guinea,maia-pila,"(PULL, TIE)"
312564,no,1215,pila1246,THOU,732,YOUR (SINGULAR),1,Nuclear Trans New Guinea,maia-pila,"(THOU, YOUR (SINGULAR))"
312565,nvkv,1213,pila1246,YOU,2274,YOUR (PLURAL),1,Nuclear Trans New Guinea,maia-pila,"(YOU, YOUR (PLURAL))"
312566,o,2642,pila1246,HE OR SHE,2473,HIS (GENITIVE OF HE),1,Nuclear Trans New Guinea,maia-pila,"(HE OR SHE, HIS (GENITIVE OF HE))"


**BOOTSTRAP**

use the df_fam as support to choose the varieties

In [11]:
new_df_fam = df_fam.groupby(['Family']).apply(lambda x: x.sample(n=2, replace=True)).reset_index(drop = True)
# take n=2 rows for every family
new_df_fam.Family.value_counts() # check

Guahiboan                   2
Turama-Kikori               2
Matacoan                    2
Itonama                     2
Abun                        2
                           ..
Sentanic                    2
Otomanguean                 2
Quechuan                    2
Nuclear Trans New Guinea    2
Dibiyaso                    2
Name: Family, Length: 185, dtype: int64

In [12]:
# create list with varieties chosen
listVar=new_df_fam.variety.tolist()

In [13]:
boot_df = pd.DataFrame()

for v in listVar:
  # for every variety, take all colexification and put them into boot_df
  boot_df=boot_df.append(df_colex_ok.loc[df_colex_ok['variety'] == v], ignore_index = True)


In [14]:
boot_df

,clics_form,Concepticon_ID.x,Glottocode,Concepticon_Gloss.x,Concepticon_ID.y,Concepticon_Gloss.y,colex,Family,variety,Concepticon_pair
0,akwac,2615,abkh1244,FLESH OR MEAT,634,MEAT,1,Abkhaz-Adyge,Abkhaz,"(FLESH OR MEAT, MEAT)"
1,az@,2615,abkh1244,FLESH OR MEAT,948,WATER,1,Abkhaz-Adyge,Abkhaz,"(FLESH OR MEAT, WATER)"
2,az@,2615,abkh1244,FLESH OR MEAT,634,MEAT,1,Abkhaz-Adyge,Abkhaz,"(FLESH OR MEAT, MEAT)"
3,az@,2615,abkh1244,FLESH OR MEAT,1007,COW,1,Abkhaz-Adyge,Abkhaz,"(FLESH OR MEAT, COW)"
4,az@,2615,abkh1244,FLESH OR MEAT,1229,OLD,1,Abkhaz-Adyge,Abkhaz,"(FLESH OR MEAT, OLD)"
...,...,...,...,...,...,...,...,...,...,...
67756,koyaci,337,zuni1245,PITCHFORK,409,FORKED BRANCH,1,Zuni,Zuni,"(PITCHFORK, FORKED BRANCH)"
67757,asa,2575,zuni1245,DO OR MAKE,1840,BUILD,1,Zuni,Zuni,"(DO OR MAKE, BUILD)"
67758,ciya,1432,zuni1245,CUT,1735,TEAR (SHRED),1,Zuni,Zuni,"(CUT, TEAR (SHRED))"
67759,hecu,1913,zuni1245,PRESS,1414,SQUEEZE,1,Zuni,Zuni,"(PRESS, SQUEEZE)"


In [15]:
# check
boot_df.groupby(['Family','variety']).size()

Family        variety              
Abkhaz-Adyge  Abkhaz                   580
Abun          Abun                       4
              Abun (Senopi Dialect)      4
Afro-Asiatic  Dera                      18
              Mandara                   20
                                      ... 
Yukaghir      Northern Yukaghir        122
              Southern Yukaghir        132
Yámana        Yagán                    406
Zamucoan      Ayoreo                   500
Zuni          Zuni                     378
Length: 269, dtype: int64

In [16]:
# check vs df_colex_ok
df_colex_ok.groupby(['Family','variety']).size()

Family        variety          
Abkhaz-Adyge  Abaza                  6
              Abkhaz               290
              Adyghe               201
              Kabardian              7
              Ubykh                  4
                                  ... 
Yukaghir      Northern Yukaghir    122
              Southern Yukaghir    132
Yámana        Yagán                203
Zamucoan      Ayoreo               250
Zuni          Zuni                 189
Length: 2787, dtype: int64

In [17]:
#df_mean = pd.DataFrame() # dataframe with family and mean per family
#df_sum = pd.DataFrame(columns=['fam', 'col', 'rep'])
suppDf = []

def concept_sum(x):
  # x = df divided by concepticon inside only one family

  #totLang = len(set(x['variety'])) # how many languages have that colexification --> SET
  totLang = len(x['variety']) # how many languages have that colexification faking to have different families from bootstrap --> NO SET
  
  newEl = [x['Family'].iloc[0], x['Concepticon_pair'].iloc[0], totLang]
  suppDf.append(newEl)

def colex_mean(x):
  # x = df divided by family
  x.groupby('Concepticon_pair').apply(lambda y: concept_sum(y))

In [18]:
# 1: how many times COL1 repetes in FAM1, FAM2...
# 2: sum of all the times for COL1
# 3: divide the sum by the tot FAMs

In [19]:
# not very clear: 
# n of languages that shows that colexification divided by total number of languages --> percentage
# mean per family: sum of number of languages that shows that colexification divided by total number of colexifications --> x languages per colexification

In [20]:
boot_df.groupby(['Family']).apply(lambda x: colex_mean(x))

""


In [21]:

df_sum = pd.DataFrame(suppDf, columns=['fam', 'col', 'rep'])

In [22]:
df_sum

,fam,col,rep
0,Abkhaz-Adyge,"(ALL, FULL)",2
1,Abkhaz-Adyge,"(APPLE, BARK)",2
2,Abkhaz-Adyge,"(APPLE, BEESWAX)",2
3,Abkhaz-Adyge,"(APPLE, LEATHER)",2
4,Abkhaz-Adyge,"(AREA, PLACE (POSITION))",2
...,...,...,...
48523,Zuni,"(WIND, BREATH OR BREATHE)",2
48524,Zuni,"(WING, SILVER)",2
48525,Zuni,"(WOOD, FIREWOOD)",2
48526,Zuni,"(YES, YOUNG WOMAN)",2


segunda version: colexification frequency within each family

In [29]:
df_sum['mean']=df_sum['rep'].apply(lambda x: x/2) # divided by 2 because we took only 2 varieties --> change if necessary

In [30]:
df_sum

,fam,col,rep,mean
0,Abkhaz-Adyge,"(ALL, FULL)",2,1.0
1,Abkhaz-Adyge,"(APPLE, BARK)",2,1.0
2,Abkhaz-Adyge,"(APPLE, BEESWAX)",2,1.0
3,Abkhaz-Adyge,"(APPLE, LEATHER)",2,1.0
4,Abkhaz-Adyge,"(AREA, PLACE (POSITION))",2,1.0
...,...,...,...,...
48523,Zuni,"(WIND, BREATH OR BREATHE)",2,1.0
48524,Zuni,"(WING, SILVER)",2,1.0
48525,Zuni,"(WOOD, FIREWOOD)",2,1.0
48526,Zuni,"(YES, YOUNG WOMAN)",2,1.0


In [34]:
df_sum[df_sum['fam']=='Afro-Asiatic']

,fam,col,rep,mean
286,Afro-Asiatic,"(ARM, FRIEND)",1,0.5
287,Afro-Asiatic,"(ATOP, SEIZE)",1,0.5
288,Afro-Asiatic,"(AXE, SICKLE)",1,0.5
289,Afro-Asiatic,"(BAOBAB TREE, CHAMELEON)",1,0.5
290,Afro-Asiatic,"(BITE, SHOOT)",1,0.5
...,...,...,...,...
372,Afro-Asiatic,"(WHEN, MAN)",1,0.5
373,Afro-Asiatic,"(WILD ANIMAL, FLESH)",1,0.5
374,Afro-Asiatic,"(WILD ANIMAL, MEAT)",1,0.5
375,Afro-Asiatic,"(WILD CAT, CAT)",1,0.5


primera version: is it correct? we don't think so, but what is it then??

### how frequent is colexification among all the families

In [23]:
len(set(df_colex_ok['Family']))

186

In [24]:
suppDfMean = []
def colex_mean2(x):
  # x = dataframe with only one colexification, different families
  med = x['rep'].sum() / 186 # being 186 the number of families
  suppDfMean.append([x['col'].iloc[0], med])

In [25]:
df_sum.groupby('col').apply(lambda x: colex_mean2(x))

""


In [26]:
df_mean = pd.DataFrame(suppDfMean, columns=['col', 'mean'])
# mean of languages per family

In [27]:
df_mean
# on average, how many languages per family show that colexification

,col,mean
0,"(ABOVE, BELOW OR UNDER)",0.005376
1,"(ABOVE, CART)",0.005376
2,"(ABOVE, CIRCLE)",0.005376
3,"(ABOVE, DEER)",0.005376
4,"(ABOVE, DIG)",0.010753
...,...,...
35217,"(YOUNGER SISTER, YOUNGER BROTHER)",0.026882
35218,"(YOUNGER SISTER, YOUNGER SIBLING)",0.021505
35219,"(YOUNGER SISTER (OF WOMAN), YOUNGER BROTHER)",0.021505
35220,"(YOUNGER SISTER (OF WOMAN), YOUNGER SIBLING)",0.021505


In [28]:
# mean of means:
df_mean['mean'].mean()

0.010343150633489393